In [ ]:
# from deap import base, creator, tools, algorithms
from dedalus import public as de
from dedalus.extras.plot_tools import plot_bot_2d
from dedalus.extras import *
import numpy as np
import scipy
from scipy import integrate, interpolate
import pickle
from math import sqrt
import matplotlib.pyplot as plt
import pandas as pd


import time as timeski
import os

#Suppress most Dedalus output
de.logging_setup.rootlogger.setLevel('ERROR')

from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42
rcParams['font.family'] = 'arial'
# rcParams['font.sans-serif'] = ['Tahoma']
rcParams.update({'font.size': 12})

In [ ]:
###################################################################
# Simulation Functions
###################################################################

#Scale list between 0 and 1
def scale(A):
    return (A-np.min(A))/(np.max(A) - np.min(A))

# steady state ODE
def ssODE(y,t,params):
    u,v,g,G = y
    b,gamma,n,sigma,e,c,d= params
    K = 1
    
    du = b*v+v*gamma*u**n/(K**n+u**n)-sigma*u-e*G*u
    dv = -du

    dG = c*u*g-d*G
    dg = - dG
    
    derivs = [du,dv,dg,dG]
    return derivs

#determine homogenous SS using ssODE
def homogenousSS(u,v,g,G,params):

    y0 = (u,v,g,G)
    t_sim = (0,1000)
    odeSoln = integrate.odeint(ssODE,y0,t_sim,args=(params,),mxstep=1000000) 
      
    return(odeSoln[1])

def exp_decay_gradient(p_ss,p_p,p_m,rs,mu_F):
    p_max = np.max([p_ss,p_p])
    p_min = np.min([p_ss,p_p])
    
    increased = False
    if p_max == p_p: 
        increased = True
        ps = [p_max if r < mu_F else p_min+(p_max-p_min)*np.exp(-p_m*(r-mu_F)) for r in rs]
    
    else: ps = (p_max+p_min)*np.ones(len(rs))-[p_max if r < mu_F else p_min+(p_max-p_min)*np.exp(-p_m*(r-mu_F)) for r in rs ]
    
    return(ps)

def WPGAP_2D(params):
#     b_SS,b_p, b_m, c_SS,c_p,c_m,delta,gamma_SS,gamma_p,gamma_m,e,d = params
    [b,c,sigma,e,gamma,d] = params

    x_basis=de.Fourier('x',128,interval=(0,8),dealias=3/2)
    y_basis=de.Chebyshev('y',128,interval=(0,8),dealias=3/2)
    #Domain:bases,datatype
    domain=de.Domain([x_basis,y_basis],float)

    x,y = domain.grids(scales=1)


    T = 4.04 
    Tg = 10
    n = 2
    K = 1

    Du = .004
    Dv = 100*Du
    DG = 100*Du #40*Du
    Dg = 100*Du

    p_ss = [b,gamma,n,sigma,e,c,d]
    u0,v0,g0,G0 = homogenousSS(T/2,T/2, Tg/2, Tg/2,p_ss)


    # Specify problem
    problem = de.IVP(domain, variables=['u', 'v','uy','vy','G','g','Gy','gy'])
    
    problem.parameters['u0'] = u0
    problem.parameters['v0'] = v0
    problem.parameters['G0'] = G0
    problem.parameters['g0'] = g0
    problem.parameters['Tg'] = Tg
    
    problem.parameters['gamma'] = gamma
    problem.parameters['b'] = b
    problem.parameters['n'] = n
    problem.parameters['c'] = c
    problem.parameters['dd'] = d
    problem.parameters['sigma'] = sigma
    problem.parameters['e'] = e
    problem.parameters['K'] = K

    problem.parameters['Du'] = Du
    problem.parameters['Dv'] = Dv

    problem.parameters['DG'] = DG
    problem.parameters['Dg'] = Dg
    
    problem.substitutions['f(u,v,G)'] = 'b*v+v*gamma*u**n/(K**n+u**n)-sigma*u-e*G*u'
    problem.substitutions['fg(u,G,g)'] = 'c*u*g-dd*G'
    
    # Mass always conserved
    problem.substitutions['minf(u,v,G)'] = '-f(u,v,G)'
    problem.substitutions['minfg(u,G,g)'] = '-fg(u,G,g)'

    problem.add_equation("dt(u)-Du*dx(dx(u))-Du*dy(uy)=f(u,v,G)")
    problem.add_equation("dt(v)-Dv*dx(dx(v))-Dv*dy(vy)=minf(u,v,G)")
    problem.add_equation("dt(G)-DG*dx(dx(G))-DG*dy(Gy)=fg(u,G,g)")
    problem.add_equation("dt(g)-Dg*dx(dx(g))-Dg*dy(gy)=minfg(u,G,g)")

    problem.add_equation("uy-dy(u)=0")
    problem.add_equation("vy-dy(v)=0")

    problem.add_equation("Gy-dy(G)=0")
    problem.add_equation("gy-dy(g)=0")

    # # Neumann boundary conditions
    problem.add_bc("left (uy) = 0")
    problem.add_bc("right (uy) = 0")
    problem.add_bc("left (vy) = 0")
    problem.add_bc("right (vy) = 0")
    problem.add_bc("left (Gy) = 0")
    problem.add_bc("right (Gy) = 0")
    problem.add_bc("left (gy) = 0")
    problem.add_bc("right (gy) = 0")

    # Pick a timestepper
    ts = de.timesteppers.RK443 #RK222
    # Build solver
    solver = problem.build_solver(ts)
    # Set integration limits
    solver.stop_wall_time = np.inf
    solver.stop_sim_time = np.inf
    solver.stop_iteration = np.inf
    # Set initial conditions
    u = solver.state ['u']
    v = solver.state['v']
    G = solver.state ['G']
    g = solver.state['g']
    
    urand = 0.1*v0*np.random.rand(*u['g'].shape)
    u['g'] = u0+urand
    v['g'] = v0-urand
    G['g'] = G0*np.ones(G['g'].shape)
    g['g'] = g0*np.ones(g['g'].shape)

    solver.stop_iteration = 10000

    dt =  0.25
    nonan = True
    not_steady = True
    prev_state = np.zeros((128*3//2,128*3//2))
    # Main loop chceking stopping criteria
    while solver.ok and nonan and not_steady:
        # Step forward
        solver.step(dt)

        if solver.iteration % 50 == 0:
            if np.count_nonzero(np.isnan(u['g'])) > 0 or np.min(u['g']) < 0 :
                return('Numerical Error')
                nonan = False  
                
        if solver.iteration% 50 ==0:
            curr_state = np.array(u['g'])
            if np.max(np.abs(curr_state-prev_state)) < 10e-4:
                print(np.max(np.abs(curr_state-prev_state)))
                print('Steady state at t = %.2f'%(np.round(solver.iteration*dt,2)))
                not_steady = False
            else: prev_state = np.array(u['g'])

    
    return u['g'].T


In [ ]:
x_basis=de.Fourier('x',128,interval=(0,8),dealias=3/2)
y_basis=de.Chebyshev('y',128,interval=(0,8),dealias=3/2)
#Domain:bases,datatype
domain=de.Domain([x_basis,y_basis],float)


In [ ]:
# ps = [0.1,1,1,1,2,1] #b,c,sigma,e,gamma,d

us_c = []
for c in [0.25,0.5,0.7,1,1.25]:
    ps = [0.1,c,1,1,2,1]
    u = WPGAP_2D(ps)
    us_c.append(u)

In [ ]:
# pickle.dump(us_c,open('GridSearchSims/C_Sims.pickle','wb'))

In [ ]:
# define original grid used by Dedalus, will interpolate (same # grid points) to plot on an evenly spaced grid

x_basis=de.Fourier('x',128,interval=(0,8),dealias=3/2)
y_basis=de.Chebyshev('y',128,interval=(0,8),dealias=3/2)
#Domain:bases,datatype
domain=de.Domain([x_basis,y_basis],float)
x,y = domain.grids(scales=domain.dealias)
X,Y = np.meshgrid(x,y)
grid_x, grid_y = np.mgrid[0:8:128j, 0:8:128j]
original_grid = np.vstack([X.flatten(),Y.flatten()]).T

In [ ]:
plt.figure(figsize=(4,1.5))


for i,c in enumerate([0.25,0.5,0.7,1,1.25]):
    plt.subplot(1,len(us_c),i+1)
    u = us_c[i]
    grid_z0 = interpolate.griddata(original_grid, u.flatten(), (grid_x, grid_y), method='nearest')
    plt.imshow(grid_z0,extent=[0,8,0,8],vmin= 0.2, vmax = 0.8)
    if i == 0:
        plt.title('$c$ = %.2f'%c)
        plt.yticks([0,8])
        plt.ylabel('$\mu m$')
    else:
        plt.title('%.2f'%c)
        plt.yticks([])
        
    plt.xticks([0,8])
    plt.subplots_adjust(wspace=0.3)
    plt.xlabel('$\mu m$')
#     plt.colorbar()
#     plt.savefig('Figures/Figure2/Parameter_Sweep_C.pdf',dpi=300,bbox_inches='tight')

In [ ]:
us_d = []
for d in [0.5,1,1.5,2,2.5]:
    ps = [0.1,1,1,1,2,d]
    u = WPGAP_2D(ps)
    us_d.append(u)

In [ ]:
# pickle.dump(us_d,open('GridSearchSims/D_Sims.pickle','wb'))

In [ ]:
plt.figure(figsize=(4,1.5))
x,y = domain.grids(scales=domain.dealias)
X,Y = np.meshgrid(x,y)
grid_x, grid_y = np.mgrid[0:8:128j, 0:8:128j]
original_grid = np.vstack([X.flatten(),Y.flatten()]).T

for i,d in enumerate([0.5,1,1.5,2,2.5]):
    plt.subplot(1,len(us_d),i+1)
    u = us_d[i]
    grid_z0 = interpolate.griddata(original_grid, u.flatten(), (grid_x, grid_y), method='nearest')
    plt.imshow(grid_z0,extent=[0,8,0,8],vmin= 0.2, vmax = 0.8)
    if i == 0:
        plt.title('$d$ = %.2f'%d)
        plt.yticks([0,8])
        plt.ylabel('$\mu m$')
    else:
        plt.title('%.2f'%d)
        plt.yticks([])
        
    plt.xticks([0,8])
    plt.xlabel('$\mu m$')
    plt.subplots_adjust(wspace=0.3)
#     plt.savefig('Figures/Figure2/Parameter_Sweep_D.pdf',dpi=300,bbox_inches='tight')

In [ ]:
us_b = []
for b in [0.01,0.05,0.1,0.2,0.3]:
    ps = [b,1,1,1,2,1]
    u = WPGAP_2D(ps)
    us_b.append(u)

In [ ]:
# pickle.dump(us_b,open('GridSearchSims/b_Sims.pickle','wb'))

In [ ]:
plt.figure(figsize=(4,1.5))
x,y = domain.grids(scales=domain.dealias)
X,Y = np.meshgrid(x,y)
grid_x, grid_y = np.mgrid[0:8:128j, 0:8:128j]
original_grid = np.vstack([X.flatten(),Y.flatten()]).T

for i,b in enumerate([0.01,0.05,0.1,0.2,0.3]):
    plt.subplot(1,len(us_b),i+1)
    u = us_b[i]
    grid_z0 = interpolate.griddata(original_grid, u.flatten(), (grid_x, grid_y), method='nearest')
    plt.imshow(grid_z0,extent=[0,8,0,8],vmin= 0.2, vmax = 0.8)
    if i == 0:
        plt.title('$b$ = %.2f'%b)
        plt.yticks([0,8])
        plt.ylabel('$\mu m$')
    else:
        plt.title('%.2f'%b)
        plt.yticks([])
        
    plt.xticks([0,8])
#     plt.xlabel('$\mu m$')
    plt.subplots_adjust(wspace=0.3)
#     plt.tight_layout()
#     plt.colorbar()
#     plt.savefig('Figures/Figure2/Parameter_Sweep_b.pdf',dpi=300,bbox_inches='tight')

In [ ]:
us_sig = []
for i,sigma in enumerate([0.01,0.1,0.5,1,1.5]):
    ps = [0.1,1,sigma,1,2,1]
    u = WPGAP_2D(ps)
    us_sig.append(u)

In [ ]:
# pickle.dump(us_sig,open('GridSearchSims/sig_Sims.pickle','wb'))

In [ ]:
plt.figure(figsize=(4,1.5))
x,y = domain.grids(scales=domain.dealias)
X,Y = np.meshgrid(x,y)
grid_x, grid_y = np.mgrid[0:8:128j, 0:8:128j]
original_grid = np.vstack([X.flatten(),Y.flatten()]).T

for i,sigma in enumerate([0.01,0.1,0.5,1,1.5]):
    plt.subplot(1,len(us_sig),i+1)
    u = us_sig[i]
    grid_z0 = interpolate.griddata(original_grid, u.flatten(), (grid_x, grid_y), method='nearest')
    plt.imshow(grid_z0,extent=[0,8,0,8],vmin= 0.2, vmax = 0.8)
    if i == 0:
        plt.title('$\sigma$ = %.2f'%sigma)
        plt.yticks([0,8])
        plt.ylabel('$\mu m$')
    else:
        plt.title('%.2f'%sigma)
        plt.yticks([])
        
    plt.xticks([0,8])
    plt.subplots_adjust(wspace=0.3)
#     plt.savefig('Figures/Figure2/Parameter_Sweep_sigma.pdf',dpi=300,bbox_inches='tight')

In [ ]:
us_e = []
for i,e in enumerate([0.5,0.6,0.75,1,1.5]):
    ps = [0.1,1,1,e,2,1]
    u = WPGAP_2D(ps)
    us_e.append(u)

In [ ]:
# pickle.dump(us_e,open('GridSearchSims/e_Sims.pickle','wb'))

In [ ]:
plt.figure(figsize=(4,1.5))
x,y = domain.grids(scales=domain.dealias)
X,Y = np.meshgrid(x,y)
grid_x, grid_y = np.mgrid[0:8:128j, 0:8:128j]
original_grid = np.vstack([X.flatten(),Y.flatten()]).T

for i,e in enumerate([0.5,0.6,0.75,1,1.5]):
    plt.subplot(1,len(us_e),i+1)
    u = us_e[i]
    grid_z0 = interpolate.griddata(original_grid, u.flatten(), (grid_x, grid_y), method='nearest')
    plt.imshow(grid_z0,extent=[0,8,0,8],vmin= 0.2, vmax = 0.8)
    if i == 0:
        plt.title('$e$ = %.2f'%e)
        plt.yticks([0,8])
        plt.ylabel('$\mu m$')
    else:
        plt.title('%.2f'%e)
        plt.yticks([])
        
    plt.xticks([0,8])
    plt.subplots_adjust(wspace=0.3)
#     plt.savefig('Figures/Figure2/Parameter_Sweep_e.pdf',dpi=300,bbox_inches='tight')

In [ ]:
us_gam = []
for i,gam in enumerate([1.5,2,2.5,3,5]):
    ps = [0.1,1,1,1,gam,1]
    u = WPGAP_2D(ps)
    us_gam.append(u)

In [ ]:
# pickle.dump(us_gam,open('GridSearchSims/gamma_Sims.pickle','wb'))

In [ ]:
plt.figure(figsize=(4,1.5))
x,y = domain.grids(scales=domain.dealias)
# # phi = np.vstack((phi,2*np.pi))
X,Y = np.meshgrid(x,y)
# x,y = domain.grids(scales=1)
grid_x, grid_y = np.mgrid[0:8:128j, 0:8:128j]
original_grid = np.vstack([X.flatten(),Y.flatten()]).T

for i,gam in enumerate([1.5,2,2.5,3,5]):
    plt.subplot(1,len(us_gam),i+1)
    u = us_gam[i]
    grid_z0 = interpolate.griddata(original_grid, u.flatten(), (grid_x, grid_y), method='nearest')
    plt.imshow(grid_z0,extent=[0,8,0,8],vmin= 0.2, vmax = 0.8)
    if i == 0:
        plt.title('$\gamma$ = %.2f'%gam)
        plt.yticks([0,8])
        plt.ylabel('$\mu m$')
    else:
        plt.title('%.2f'%gam)
        plt.yticks([])
        
    plt.xticks([0,8])
    plt.subplots_adjust(wspace=0.3)

#     plt.savefig('Figures/Figure2/Parameter_Sweep_gamma.pdf',dpi=300,bbox_inches='tight')